In [64]:
import os
import pandas as pd
from difflib import SequenceMatcher
import numpy as np

# Methods for geocode mappings

In [65]:
geograph_df = pd.read_csv('geo.csv')

# Remove spaces before the column names
geograph_df.columns = geograph_df.columns.str.strip()

In [66]:
def add_geocode_level(df, geo_code, geo_level):
    df['geo_code'] = geo_code
    df['geo_level'] = geo_level
    return df

In [92]:
# Find the geo_code from the name of location
def get_code(name, level='district', parent_code=None):
    try:
        check_df = geograph_df.copy()
        if parent_code:
            # Search only over portion with given parent code
            check_df = check_df[check_df.parent_code == parent_code]
        # Search using name of location and level
        out = check_df[(check_df['name'] == name) & (check_df['geo_level'] == level)].iloc[0]['geo_code']
        return out
    except Exception as err:
        print("Error when getting get_code", err)
        print(name, level, parent_code)
        raise

In [68]:
def local_level_name_fixes(name):
    name = name.replace(' Rural Municipality', '').replace(' Municipality', '')
    name = name.replace(' Metropolitan', '')
    name = name.replace(' Sub-Metropolitan City', '')
    return name.strip()

In [69]:
# Manual created mapping from national data to NepalMap names
# (district_code, local_level_name) -> nepalmap name
local_level_mapping = {('01', 'Panchdeval Binayak Municipality'): 'Panchadewal Binayak',
 ('01', 'Turmakhand Rural Municipality'): 'Turmakhad',
 ('11', 'Aamchok Rural Municipality'): 'Aamchowk',
 ('11', 'Sadananda Municipality'): 'Shadananda',
 ('11', 'Temkemaiyung Rural Municipality'): 'Tyamkemaiyung',
 ('12', 'Ichchhakamana Rural Municipality'): 'Ichchhyakamana',
 ('13', 'Aalital Rural Municipality'): 'Alital',
 ('13', 'Ajayameru Rural Municipality'): 'Ajaymeru',
 ('13', 'Bhageshwor Rural Municipality'): 'Bhageshwar',
 ('13', 'Ganyapdhura Rural Municipality'): 'Ganayapdhura',
 ('13', 'Navadurga Rural Municipality'): 'Nawadurga',
 ('14', 'Aathbis Municipality'): 'Aathabis',
 ('14', 'Bhairavi Rural Municipality'): 'Bhairabi',
 ('14', 'Bindrasaini Municipality'): 'Chamunda Bindrasaini',
 ('15', 'Bangalachuli Rural Municipality'): 'Banglachuli',
 ('16', 'Duhun Rural Municipality'): 'Dunhu',
 ('16', 'Malikarjun Rural Municipality'): 'Malikaarjun',
 ('16', 'Vyans Rural Municipality'): 'Byas',
 ('17', 'Galchhi Rural Municipality'): 'Galchi',
 ('17', 'Khaniyabas Rural Municipality'): 'Khaniyabash',
 ('17', 'Netrawati Dabjong Rural Municipality'): 'Netrawati',
 ('17', 'Nilkantha Municipality'): 'Nilakantha',
 ('17', 'Rubivalley Rural Municipality'): 'Rubi Valley',
 ('17', 'Tripurasundari Rural Municipality'): 'Tripura Sundari',
 ('18', 'Shahidbhumi Rural Municipality'): 'Khalsa Chhintang Shahidbhumi',
 ('19', 'Aurahi Rural Municipality'): 'Aaurahi',
 ('19', 'Janaknandini Rural Municipality'): 'Janaknandani',
 ('19', 'Janakpurdham Sub-Metropolitan City'): 'Janakpur',
 ('19', 'Kshireshwornath Municipality'): 'Chhireshwornath',
 ('19', 'Laxminiya Rural Municipality'): 'Lakshminiya',
 ('19',
  'Mukhiyapatti Musaharmiya Rural Municipality'): 'Mukhiyapatti Musarmiya',
 ('19', 'Shahidnagar Municipality'): 'Sahidnagar',
 ('02', 'Bhumikasthan Municipality'): 'Bhumekasthan',
 ('02', 'Shitganga Municipality'): 'Sitganga',
 ('20', 'Shailung Rural Municipality'): 'Sailung',
 ('21', 'Dolpobuddha Rural Municipality'): 'Dolpo Buddha',
 ('21', 'Thulibheri Municipality'): 'Thuli Bheri',
 ('22', 'Aadarsha Rural Municipality'): 'Adharsha',
 ('22', 'Bogatan Phudsil Rural Municipality'): 'Bogtan',
 ('22', 'Dipayal Silgadhi Municipality'): 'Dipayal Silgadi',
 ('22', 'K.I. Singh Rural Municipality'): 'K I Singh',
 ('23', 'Barpak Sulikot Rural Municipality'): 'Sulikot',
 ('23', 'Bhimsen Thapa Rural Municipality'): 'Bhimsen',
 ('23', 'Chumnubri Rural Municipality'): 'Chum Nubri',
 ('23', 'Shahid Lakhan Rural Municipality'): 'Sahid Lakhan',
 ('24', 'Chhatrakot Rural Municipality'): 'Chatrakot',
 ('26', 'Ilam Municipality'): 'Illam',
 ('26', 'Phakphokthum Rural Municipality'): 'Fakphokthum',
 ('27', 'Kushe Rural Municipality'): 'Kuse',
 ('27', 'Nalgad Municipality'): 'Tribeni Nalagad',
 ('27', 'Shivalaya Rural Municipality'): 'Shiwalaya',
 ('28', 'Bahradashi Rural Municipality'): 'Barhadashi',
 ('28', 'Gauradaha Municipality'): 'Gauradhaha',
 ('28', 'Gaurigunj Rural Municipality'): 'Gauriganj',
 ('28', 'Shivasatakshi Municipality'): 'Shivasataxi',
 ('29', 'Patarasi Rural Municipality'): 'Patrasi',
 ('03', 'Jaimini Municipality'): 'Jaimuni',
 ('03', 'Kathekhola Rural Municipality'): 'Kanthekhola',
 ('03', 'Tamankhola Rural Municipality'): 'Taman Khola',
 ('03', 'Tarakhola Rural Municipality'): 'Tara Khola',
 ('30', 'Bardgoriya Rural Municipality'): 'Bardagoriya',
 ('31', 'Pachaljharna Rural Municipality'): 'Pachaljharana',
 ('31', 'Sanni Triveni Rural Municipality'): 'Sanni Tribeni',
 ('31', 'Shuvakalika Rural Municipality'): 'Kalika',
 ('31', 'Tilagupha Municipality'): 'Tilagufa',
 ('33', 'Shuddhodan Rural Municipality'): 'Suddhodhan',
 ('34', 'Pokhara Metropolitan'): 'Pokhara Lekhnath',
 ('35', 'Budhanilkantha Municipality'): 'Budhanilakantha',
 ('35', 'Kageshwori Manohara Municipality'): 'Kageshwori Manahora',
 ('36', 'Bethanchok Rural Municipality'): 'Bethanchowk',
 ('36', 'Chauri Deurali Rural Municipality'): 'Chaurideurali',
 ('36', 'Mandan Deupur Municipality'): 'Mandandeupur',
 ('37', 'Ainselukharka Rural Municipality'): 'Ainselukhark',
 ('37', 'Diktel Rupakot Majhuwagadhi Municipality'): 'Rupakot Majhuwagadhi',
 ('37', 'Diprung Chuichumma Rural Municipality'): 'Diprung',
 ('37', 'Kepilasgadhi Rural Municipality'): 'Kepilasagadhi',
 ('37', 'Rawabesi Rural Municipality'): 'Lamidanda', #fixed
 ('04', 'Dasharath Chand Municipality'): 'Dasharathchanda',
 ('04', 'Pancheshwor Rural Municipality'): 'Pancheshwar',
 ('40', 'Balawa Municipality'): 'Balwa',
 ('40', 'Ekdara Rural Municipality'): 'Ekdanra',
 ('40', 'Jaleshwor Municipality'): 'Jaleswor',
 ('40', 'Manara Shiswa Municipality'): 'Manra Siswa',
 ('40', 'Pipara Rural Municipality'): 'Pipra',
 ('40', 'Sonma Rural Municipality'): 'Sonama',
 ('41', 'Hetaunda Sub-Metropolitan City'): 'Hetauda',
 ('41', 'Makwanpurgadhi Rural Municipality'): 'Makawanpurgadhi',
 ('42', 'Manang Ngisyang Rural Municipality'): 'Neshyang',
 ('42', 'Narpabhumi Rural Municipality'): 'Narphu',
 ('42', 'Nason Rural Municipality'): 'Nashong',
 ('43', 'Pathari Shanishchare Municipality'): 'Patahrishanishchare',
 ('43', 'Sunbarsi Municipality'): 'Sunwarshi',
 ('43', 'Sundarharaincha Municipality'): 'Sundarharaicha',
 ('43', 'Urlabari Municipality'): 'Uralabari',
 ('45', 'Baragung Muktikshetra Rural Municipality'): 'Barhagaun Muktikhsetra',
 ('45', 'Gharpajhong Rural Municipality'): 'Gharapjhong',
 ('45', 'Lo-Ghekar Damodarkunda Rural Municipality'): 'Dalome', # fix
 ('46', 'Dhawalagiri Rural Municipality'): 'Dhaulagiri',
 ('47', 'Baudikali Rural Municipality'): 'Bungdikali',
 ('47', 'Binayi Triveni Rural Municipality'): 'Binayee Tribeni',
 ('47', 'Gaindakot Municipality'): 'Gaidakot',
 ('48', 'Dupcheshwor Rural Municipality'): 'Dupcheshwar',
 ('48', 'Myagang Rural Municipality'): 'Meghang',
 ('48', 'Tarakeshwor Rural Municipality'): 'Tarkeshwar',
 ('49', 'Chishankhugadhi Rural Municipality'): 'Chisankhugadhi',
 ('05', 'Bitthadchir Rural Municipality'): 'Bithadchir',
 ('05', 'Chhabis Pathibhera Rural Municipality'): 'Chabispathivera',
 ('05', 'Jayaprithvi Municipality'): 'Jayaprithivi',
 ('05', 'Kedarsyun Rural Municipality'): 'Kedarseu',
 ('05', 'Saipal Rural Municipality'): 'Kanda', #fix
 ('50', 'Baganaskali Rural Municipality'): 'Bagnaskali',
 ('51', 'Phalelung Rural Municipality'): 'Falelung',
 ('51', 'Phalgunanda Rural Municipality'): 'Falgunanda',
 ('52', 'Paiyun Rural Municipality'): 'Painyu',
 ('53', 'Bahudarmai Municipality'): 'Bahudaramai',
 ('53', 'Pakaha Mainpur Rural Municipality'): 'Pakahamainpur',
 ('53', 'Paterwa Sugauli Rural Municipality'): 'Paterwasugauli',
 ('53', 'Sakhuwa Prasauni Rural Municipality'): 'Sakhuwaprasauni',
 ('54', 'Airawati Rural Municipality'): 'Ayirabati',
 ('54', 'Swargadwari Municipality'): 'Sworgadwary',
 ('55', 'Khandadevi Rural Municipality'): 'Khadadevi',
 ('55', 'Likhu Tamakoshi Rural Municipality'): 'Likhu',
 ('56', 'Aamachhodingmo Rural Municipality'): 'Parbati Kunda',
 ('56', 'Gosainkunda Rural Municipality'): 'Gosaikunda',
 ('57', 'Dewahi Gonahi Municipality'): 'Dewahhi Gonahi',
 ('57', 'Durgabhagawati Rural Municipality'): 'Durga Bhagwati',
 ('57', 'Ishnath Municipality'): 'Ishanath',
 ('57', 'Madhavnarayan Municipality'): 'Madhav Narayan',
 ('57', 'Yamunamai Rural Municipality'): 'Yemunamai',
 ('58', 'Gangadev Rural Municipality'): 'Sukidaha',
 ('58', 'Parivartan Rural Municipality'): 'Duikholi', #fix
 ('58', 'Runtigadhi Rural Municipality'): 'Runtigadi',
 ('58', 'Sunil Smriti Rural Municipality'): 'Suwarnabati', #fix
 ('58', 'Triveni Rural Municipality'): 'Tribeni',
 ('06', 'Jagannath Rural Municipality'): 'Pandav Gupha', #fix
 ('06', 'Khaptad Chhededaha Rural Municipality'): 'Chhededaha',
 ('06', 'Swamikartik Khapar Rural Municipality'): 'Swami Kartik',
 ('06', 'Triveni Municipality'): 'Tribeni',
 ('60', 'Shuddhodan Rural Municipality'): 'Sudhdhodhan',
 ('60', 'Tilottama Municipality'): 'Tillotama',
 ('61', 'Kumakh Rural Municipality'): 'Kumakhmalika',
 ('61', 'Siddhakumakh Rural Municipality'): 'Dhorchaur',
 ('61', 'Triveni Rural Municipality'): 'Tribeni',
 ('62', 'Panchkhapan Municipality'): 'Panchakhapan',
 ('63',
  'Agnisair Krishnasawaran Rural Municipality'): 'Agnisair Krishna Savaran',
 ('63', 'Balanbihul Rural Municipality'): 'Balan Bihul',
 ('63', 'Bodebarsain Municipality'): 'Bode Barsain',
 ('63', 'Rajgadh Rural Municipality'): 'Belhi Chapena', #fix
 ('64', 'Brahmapuri Rural Municipality'): 'Bramhapuri',
 ('64', 'Dhanakaul Rural Municipality'): 'Dhankaul',
 ('65', 'Dudhauli Municipality'): 'Dudhouli',
 ('65', 'Ghyanglekh Rural Municipality'): 'Ghanglekh',
 ('66', 'Bahrabise Municipality'): 'Barhabise',
 ('66', 'Balephi Rural Municipality'): 'Balefi',
 ('66', 'Lisankhupakhar Rural Municipality'): 'Lisangkhu Pakhar',
 ('67', 'Bhagwanpur Rural Municipality'): 'Bhagawanpur',
 ('67', 'Navarajpur Rural Municipality'): 'Nawarajpur',
 ('67', 'Sakhuwa Nankarkatti Rural Municipality'): 'Sakhuwanankarkatti',
 ('68', 'Khumbu Pasanglhamu Rural Municipality'): 'Khumbupasanglahmu',
 ('68', 'Mapya Dudhkoshi Rural Municipality'): 'Dudhkoshi',
 ('68', 'Solu Dudhkunda Municipality'): 'Solududhakunda',
 ('68', 'Thulung Dudhkoshi Rural Municipality'): 'Dudhkaushika',
 ('69', 'Barahakshetra Municipality'): 'Barah',
 ('69', 'Bhokraha Narsingh Rural Municipality'): 'Bhokraha',
 ('69', 'Dewangunj Rural Municipality'): 'Dewanganj',
 ('69', 'Harinagar Rural Municipality'): 'Harinagara',
 ('07', 'Janaki Rural Municipality'): 'Janki',
 ('70', 'Barahatal Rural Municipality'): 'Barahtal',
 ('70', 'Lekbesi Municipality'): 'Lekbeshi',
 ('70', 'Panchapuri Municipality'): 'Panchpuri',
 ('71', 'Kaligandaki Rural Municipality'): 'Kaligandagi',
 ('72', 'Aanbukhaireni Rural Municipality'): 'Anbukhaireni',
 ('72', 'Rishing Rural Municipality'): 'Rhishing',
 ('73', 'Aathrai Triveni Rural Municipality'): 'Aathrai Tribeni',
 ('73', 'Sidingwa Rural Municipality'): 'Sidingba',
 ('74', 'Aatharai Rural Municipality'): 'Aathrai',
 ('74', 'Menchhayayem Rural Municipality'): 'Menchayam',
 ('75', 'Limchungbung Rural Municipality'): 'Sunkoshi', #fixed
 ('76', 'Palhinandan Rural Municipality'): 'Palhi Nandan',
 ('77', 'Banphikot Rural Municipality'): 'Banfikot',
 ('77', 'Sanibheri Rural Municipality'): 'Sani Bheri',
 ('77', 'Triveni Rural Municipality'): 'Tribeni',
 ('08', 'Aadarsha Kotwal Rural Municipality'): 'Adarshkotwal',
 ('08', 'Jitpur Simra Sub-Metropolitan City'): 'Jitpur Simara',
 ('08', 'Kolhawi Municipality'): 'Kolhabi',
 ('08', 'Pachrauta Municipality'): 'Pacharauta',
 ('08', 'Suvarna Rural Municipality'): 'Suwarna',
 ('09', 'Bansgadhi Municipality'): 'Bansagadhi',
 ('09', 'Madhuban Municipality'): 'Madhuwan'}

In [70]:
def get_local_geo_code(local_level, district_code):
    local_level = local_level_mapping.get((district_code, local_level), local_level)
    fixed_local_name = local_level_name_fixes(local_level)
    code = get_code(fixed_local_name, level='local', parent_code=district_code)
    return code

# Transformation functions for each data

In [71]:
# Function to aggregate data for national level
def add_national_data(all_levels_data, category_column, total_column):
    nepal_data = all_levels_data[all_levels_data.geo_level == 'province'].groupby(category_column, as_index=False)[[total_column]].sum()
    nepal_data = add_geocode_level(nepal_data, 'NP', 'country')
    combined_data = pd.concat([nepal_data, all_levels_data])
    return combined_data

In [72]:
def transform_population_by_religion(df):
    df = df[df['Population by Religion'] != 'Total']
    df = df.rename(columns={'Number': 'total', 'Population by Religion': 'religion'})
    df['religion'] = df['religion'].replace('Undefined', 'Others')
    return df

In [73]:
def transform_agricultural_land(df):
    df = df[df['Agricultural land use area'] != 'Total']
    df = df.rename(columns={'Hectare': 'total'})
    df['Agricultural land use area'] = df['Agricultural land use area'].str.replace('Area under ', '').str.title()
    return df

In [74]:
def transform_agriculture_holding(df):
    COLUMN = 'Agricultural holdings size by land tenure'
    df = df[df[COLUMN] != 'Total']
    df = df.rename(columns={'Hectare': 'total'})
    df[COLUMN] = df[COLUMN].str.replace(' Size', '')
    return df

In [75]:
def organize_by_gender(df, column, status_col):
    df = df.dropna(subset=[column]).copy()
    df[status_col] = np.nan
    current_status = None
    new_rows = []
    for lab, row in df.iterrows():
        status = row[column]
        if status not in ['Male', 'Female']:
            current_status = status
        else:
            row[status_col] = current_status
        new_rows.append(row)
    df = pd.DataFrame(new_rows).dropna()
    df = df.rename(columns={column: 'sex'})
    return df

In [76]:
def take_total_only(df, column):
    new_df = df[~df[column].isin(['Male', 'Female'])].copy()
    new_df[column] = new_df[column].ffill()
    new_df = new_df.dropna().reset_index(drop=True)
    return new_df

In [77]:
def transform_literacy_gender(df):
    df = df.rename(columns={'Number': 'total'})
    column = 'Population by Literacy Status'
    df = organize_by_gender(df, column, 'literacy')
    df['total'] = df['total'].astype(int)
    return df

In [78]:
def transform_disability(df):
    df = df.rename(columns={'Number': 'total'})
    column = 'Population by disability type'
    df = organize_by_gender(df, column, 'disability')
    df['total'] = df['total'].astype(int)
    return df

In [79]:
def tranform_education_attainment(df):
    df = df.rename(columns={'Number': 'total'})
    column = 'Population by Educational Attainment'
    df = organize_by_gender(df, column, 'education level passed')
    df['total'] = df['total'].astype(int)
    return df

In [80]:
def transform_house_ownership(df):
    COLUMN = 'Number of Households by Ownership of House'
    df = df[df[COLUMN] != 'Total']
    df = df.rename(columns={'Number': 'total', 'Number of Households by Ownership of House': 'home ownership'})
    return df

In [81]:
def transform_drinking_water(df):
    COLUMN = 'Number of Households by Source of Drinking Water'
    df = df[df[COLUMN] != 'Total']
    df = df.rename(columns={'Number': 'total', 'Number of Households by Source of Drinking Water': 'drinking water source'})
    return df

In [82]:
def transform_lightning(df):
    COLUMN = 'Number of Households by Source of Lighting'
    df = df[df[COLUMN] != 'Total']
    df[COLUMN] = df[COLUMN].replace('Not Sated about Light', 'Not Stated about Light')
    df = df.rename(columns={'Number': 'total', COLUMN: 'lighting fuel'})
    return df

In [83]:
def transform_cooking_fuel(df):
    COLUMN = 'Number of Households by Usage of Cooking Fuel'
    df = df[df[COLUMN] != 'Total']
    df[COLUMN] = df[COLUMN].replace('Not Sated about Fuel', 'Not Stated about Fuel')
    df = df.rename(columns={'Number': 'total', COLUMN: 'main type of cooking fuel'})
    return df

In [84]:
def transform_toilet(df):
    COLUMN = 'Number of households by type of toilet facility'
    df = df[df[COLUMN] != 'Total']
    df = df.rename(columns={'Number': 'total', COLUMN: 'toilet type'})
    return df

In [85]:
def transform_foundation(df):
    COLUMN = 'Number of Households by Type of House Foundation'
    df = df[df[COLUMN] != 'Total']
    df = df.rename(columns={'Number': 'total', COLUMN: 'foundation type'})
    return df

In [86]:
def transform_outer_wall(df):
    COLUMN = 'Number of households by type of outer wall'
    df = df[df[COLUMN] != 'Total']
    df = df.rename(columns={'Number': 'total', COLUMN: 'outer wall type'})
    return df

In [87]:
def transform_roof(df):
    COLUMN = 'Number of Households by Type of Roof'
    df = df[df[COLUMN] != 'Total']
    df[COLUMN] = df[COLUMN].replace('Not Sated Roof', 'Not Stated Roof')
    df = df.rename(columns={'Number': 'total', COLUMN: 'roof type'})
    return df

In [88]:
def transform_ethnicity(df):
    COLUMN = 'Population by Caste Ethinicity'
    df = take_total_only(df, COLUMN)
    df = df[df[COLUMN] != 'Total']
    df = df.rename(columns={'Number': 'total', COLUMN: 'caste or ethnic group'})
    return df

In [89]:
def transform_mother_tongue(df):
    COLUMN = 'Population by Mother tongue'
    df = take_total_only(df, COLUMN)
    df = df[df[COLUMN] != 'Total']
    df = df.rename(columns={'Number': 'total', COLUMN: 'language'})
    return df 

# Process all files

In [90]:
def transform_data_file(data_file):
    final_dfs = []
    all_districts = []
    all_local_levels = []
    # File to transformation mapping
    transform_data_function = {
        'population-by-religion.csv': transform_population_by_religion,
        'agricultural-land-use-area.csv': transform_agricultural_land,
        'agricultural-holdings-size-by-land-tenure.csv': transform_agriculture_holding,
        'population-by-literacy-status.csv': transform_literacy_gender,
        'number-of-households-by-ownership-of-house.csv': transform_house_ownership,
        'number-of-households-by-source-of-drinking-water.csv': transform_drinking_water,
        'number-of-households-by-source-of-lighting.csv': transform_lightning,
        'number-of-households-by-usage-of-cooking-fuel.csv': transform_cooking_fuel,
        'number-of-households-by-type-of-toilet-facility.csv': transform_toilet,
        'number-of-households-by-type-of-house-foundation.csv': transform_foundation,
        'number-of-households-by-type-of-outer-wall.csv': transform_outer_wall,
        'number-of-households-by-type-of-roof.csv': transform_roof,
        'population-by-educational-attainment.csv': tranform_education_attainment, 
        'population-by-caste-ethinicity.csv': transform_ethnicity,
        'population-by-mother-tongue.csv': transform_mother_tongue,
        'population-by-disability-type.csv': transform_disability,
    }.get(data_file)
    # File to column mapping, required when grouping data to find national total
    columns = {
        'population-by-religion.csv': ['religion', 'total'],
        'agricultural-land-use-area.csv': ['Agricultural land use area', 'total'],
        'agricultural-holdings-size-by-land-tenure.csv': ['Agricultural holdings size by land tenure', 'total'],
        'population-by-literacy-status.csv': [['literacy', 'sex'], 'total'],
        'population-by-educational-attainment.csv': [['education level passed', 'sex'], 'total'], 
        'number-of-households-by-ownership-of-house.csv': ["home ownership", 'total'],
        'number-of-households-by-source-of-drinking-water.csv': ['drinking water source', 'total'],
        'number-of-households-by-source-of-lighting.csv': ['lighting fuel', 'total'],
        'number-of-households-by-usage-of-cooking-fuel.csv': ['main type of cooking fuel', 'total'],
        'number-of-households-by-type-of-toilet-facility.csv': ['toilet type', 'total'],
        'number-of-households-by-type-of-house-foundation.csv': ['foundation type', 'total'],
        'number-of-households-by-type-of-outer-wall.csv': ['outer wall type', 'total'],
        'number-of-households-by-type-of-roof.csv': ['roof type', 'total'],
        'population-by-caste-ethinicity.csv': ['caste or ethnic group', 'total'],
        'population-by-mother-tongue.csv': ['language', 'total'],
        'population-by-disability-type.csv': [['disability', 'sex'], 'total'],
    }.get(data_file)
    
    provinces = os.listdir('Province')
    province_parent = os.path.join(os.getcwd(), 'Province')
    for province in provinces:
        # For each province
        province_dir = os.path.join(province_parent, province)
        province_files = os.path.join(province_dir, 'csvs')
        district_folder_path = os.path.join(province_dir, 'District')
        data_location = os.path.join(province_files, data_file)
        df = pd.read_csv(data_location)
        df = add_geocode_level(transform_data_function(df), province, 'province')
        final_dfs.append(df)
        
        # Find districts
        districts = os.listdir(district_folder_path)
        for district in districts:
            # For each district
            each_district_path = os.path.join(district_folder_path, district)
            each_district_csvs = os.path.join(each_district_path, 'csvs')
            data_location = os.path.join(each_district_csvs, data_file)
            df = pd.read_csv(data_location)
            district_code = get_code(district, level='district', parent_code=province)
            df = add_geocode_level(transform_data_function(df), get_code(district), 'district')
            final_dfs.append(df)
            local_levels_folder_path = os.path.join(each_district_path, 'LocalLevel')
            local_levels = os.listdir(local_levels_folder_path)
            for local_level in local_levels:
                # For each local level
                each_local_path = os.path.join(local_levels_folder_path, local_level)
                each_local_csvs = os.path.join(each_local_path, 'csvs')
                data_location = os.path.join(each_local_csvs, data_file)
                df = pd.read_csv(data_location)
                print(district_code, local_level)
                local_level_code = get_local_geo_code(local_level, district_code)
                df = add_geocode_level(transform_data_function(df), local_level_code, 'local')
                final_dfs.append(df)
            all_local_levels.extend([(district_code, local_level) for local_level in local_levels])
        all_districts.extend(districts)
        final_dfs.append(df)

    transformed_data = pd.concat(final_dfs)

    combined_data = add_national_data(transformed_data, *columns)

    combined_data = combined_data.reset_index(drop=True)
    combined_data.drop_duplicates(inplace=True)
    combined_data['total'] = combined_data['total'].astype(int)
    combined_data.to_csv(os.path.join('output', data_file), index=False)
    return all_local_levels

In [91]:
data_files = [
    'population-by-religion.csv',
#     'agricultural-land-use-area.csv',
#     'agricultural-holdings-size-by-land-tenure.csv',
    'number-of-households-by-ownership-of-house.csv',
    'number-of-households-by-source-of-drinking-water.csv',
    'number-of-households-by-source-of-lighting.csv',
    'number-of-households-by-usage-of-cooking-fuel.csv',
    'number-of-households-by-type-of-toilet-facility.csv',
    'number-of-households-by-type-of-house-foundation.csv',
    'number-of-households-by-type-of-outer-wall.csv',
    'number-of-households-by-type-of-roof.csv',
    'population-by-literacy-status.csv',
    'population-by-educational-attainment.csv',
    'population-by-caste-ethinicity.csv',
    'population-by-mother-tongue.csv',
    'population-by-disability-type.csv'
]

for data_file in data_files:
    all_local_levels = transform_data_file(data_file)

56 Uttargaya Rural Municipality
56 Gosainkunda Rural Municipality
56 Naukunda Rural Municipality
56 Kalika Rural Municipality
56 Aamachhodingmo Rural Municipality
65 Sunkoshi Rural Municipality
65 Phikkal Rural Municipality
65 Hariharpurgadhi Rural Municipality
65 Marin Rural Municipality
65 Dudhauli Municipality
65 Ghyanglekh Rural Municipality
65 Tinpatan Rural Municipality
65 Golanjor Rural Municipality
65 Kamalamai Municipality
66 Sunkoshi Rural Municipality
66 Lisankhupakhar Rural Municipality
66 Jugal Rural Municipality
66 Indrawati Rural Municipality
66 Chautara Sangachokgadhi Municipality
66 Bhotekoshi Rural Municipality
66 Helambu Rural Municipality
66 Balephi Rural Municipality
66 Melamchi Municipality
66 Tripurasundari Rural Municipality
66 Bahrabise Municipality
66 Panchpokhari Thangpal Rural Municipality
12 Ichchhakamana Rural Municipality
12 Rapti Municipality
12 Khairahani Municipality
12 Ratnanagar Municipality
12 Kalika Municipality
12 Bharatpur Metropolitan
12 Madi Mu

39 Besishahar Municipality
39 Marsyangdi Rural Municipality
39 Sundarbazar Municipality
39 Kwholasothar Rural Municipality
39 Dudhpokhari Rural Municipality
39 Dordi Rural Municipality
34 Rupa Rural Municipality
34 Annapurna Rural Municipality
34 Madi Rural Municipality
34 Machhapuchchhre Rural Municipality
34 Pokhara Metropolitan
42 Nason Rural Municipality
42 Chame Rural Municipality
42 Manang Ngisyang Rural Municipality
42 Narpabhumi Rural Municipality
72 Devghat Rural Municipality
72 Rishing Rural Municipality
72 Aanbukhaireni Rural Municipality
72 Bhimad Municipality
72 Bhanu Municipality
72 Bandipur Rural Municipality
72 Ghiring Rural Municipality
72 Shuklagandaki Municipality
72 Byas Municipality
72 Myagde Rural Municipality
45 Baragung Muktikshetra Rural Municipality
45 Lomanthang Rural Municipality
45 Thasang Rural Municipality
45 Lo-Ghekar Damodarkunda Rural Municipality
45 Gharpajhong Rural Municipality
52 Mahashila Rural Municipality
52 Paiyun Rural Municipality
52 Jaljala 

11 Aamchok Rural Municipality
11 Arun Rural Municipality
11 Hatuwagadhi Rural Municipality
11 Temkemaiyung Rural Municipality
11 Ramprasad Rai Rural Municipality
18 Pakhribas Municipality
18 Shahidbhumi Rural Municipality
18 Chhathar Jorpati Rural Municipality
18 Mahalaxmi Municipality
18 Dhankuta Municipality
18 Chaubise Rural Municipality
18 Sangurigadhi Rural Municipality
62 Dharmadevi Municipality
62 Bhotkhola Rural Municipality
62 Silichong Rural Municipality
62 Makalu Rural Municipality
62 Madi Municipality
62 Chainpur Municipality
62 Khandbari Municipality
62 Chichila Rural Municipality
62 Panchkhapan Municipality
62 Sabhapokhari Rural Municipality
75 Tapli Rural Municipality
75 Rautamai Rural Municipality
75 Triyuga Municipality
75 Belaka Municipality
75 Limchungbung Rural Municipality
75 Katari Municipality
75 Udayapurgadhi Rural Municipality
75 Chaudandigadhi Municipality
51 Kummayak Rural Municipality
51 Phalelung Rural Municipality
51 Hilihang Rural Municipality
51 Miklajun

48 Tadi Rural Municipality
48 Tarakeshwor Rural Municipality
48 Dupcheshwor Rural Municipality
48 Kispang Rural Municipality
48 Suryagadhi Rural Municipality
48 Myagang Rural Municipality
48 Belkotgadhi Municipality
48 Kakani Rural Municipality
17 Rubivalley Rural Municipality
17 Khaniyabas Rural Municipality
17 Galchhi Rural Municipality
17 Nilkantha Municipality
17 Siddhalek Rural Municipality
17 Gangajamuna Rural Municipality
17 Netrawati Dabjong Rural Municipality
17 Gajuri Rural Municipality
17 Jwalamukhi Rural Municipality
17 Tripurasundari Rural Municipality
17 Dhunibesi Municipality
17 Thakre Rural Municipality
17 Benighat Rorang Rural Municipality
24 Kaligandaki Rural Municipality
24 Chandrakot Rural Municipality
24 Resunga Municipality
24 Malika Rural Municipality
24 Ruru Rural Municipality
24 Satyawati Rural Municipality
24 Dhurkot Rural Municipality
24 Chhatrakot Rural Municipality
24 Gulmidarbar Rural Municipality
24 Madane Rural Municipality
24 Musikot Municipality
24 Ism

63 Hanumannagar Kankalini Municipality
63 Rajgadh Rural Municipality
63 Tirahut Rural Municipality
63 Rajbiraj Municipality
63 Agnisair Krishnasawaran Rural Municipality
63 Surunga Municipality
63 Rupani Rural Municipality
63 Tilathi Koiladi Rural Municipality
63 Shambhunath Municipality
63 Saptakoshi Municipality
63 Mahadeva Rural Municipality
63 Kanchanrup Municipality
63 Chhinnamasta Rural Municipality
57 Brindaban Municipality
57 Baudhimai Municipality
57 Durgabhagawati Rural Municipality
57 Chandrapur Municipality
57 Ishnath Municipality
57 Phatuwa Bijayapur Municipality
57 Katahariya Municipality
57 Gadhimai Municipality
57 Rajdevi Municipality
57 Rajpur Municipality
57 Yamunamai Rural Municipality
57 Garuda Municipality
57 Paroha Municipality
57 Dewahi Gonahi Municipality
57 Gujara Municipality
57 Madhavnarayan Municipality
57 Maulapur Municipality
57 Gaur Municipality
08 Prasauni Rural Municipality
08 Suvarna Rural Municipality
08 Parwanipur Rural Municipality
08 Karaiyamai Rur

...

76 Ramgram Municipality
76 Palhinandan Rural Municipality
76 Sarawal Rural Municipality
76 Bardaghat Municipality
76 Sunwal Municipality
76 Susta Rural Municipality
76 Pratappur Rural Municipality
07 Duduwa Rural Municipality
07 Narainapur Rural Municipality
07 Rapti Sonari Rural Municipality
07 Baijanath Rural Municipality
07 Khajura Rural Municipality
07 Kohalpur Municipality
07 Nepalgunj Sub-Metropolitan City
07 Janaki Rural Municipality
50 Rampur Municipality
50 Rainadevi Chhahara Rural Municipality
50 Tinau Rural Municipality
50 Nisdi Rural Municipality
50 Ribdikot Rural Municipality
50 Rambha Rural Municipality
50 Baganaskali Rural Municipality
50 Tansen Municipality
50 Mathagadhi Rural Municipality
50 Purbakhola Rural Municipality
58 Sunil Smriti Rural Municipality
58 Parivartan Rural Municipality
58 Madi Rural Municipality
58 Rolpa Municipality
58 Triveni Rural Municipality
58 Gangadev Rural Municipality
58 Thawang Rural Municipality
58 Runtigadhi Rural Municipality
58 Lungri R

19 Mithila Bihari Municipality
19 Laxminiya Rural Municipality
19 Dhanauji Rural Municipality
19 Aurahi Rural Municipality
19 Janaknandini Rural Municipality
19 Janakpurdham Sub-Metropolitan City
19 Hansapur Municipality
19 Bideha Municipality
19 Bateshwor Rural Municipality
04 Pancheshwor Rural Municipality
04 Shivanath Rural Municipality
04 Surnaya Rural Municipality
04 Sigas Rural Municipality
04 Purchaudi Municipality
04 Dogadakedar Rural Municipality
04 Melauli Municipality
04 Dasharath Chand Municipality
04 Dilasaini Rural Municipality
04 Patan Municipality
16 Mahakali Municipality
16 Shailyashikhar Municipality
16 Apihimal Rural Municipality
16 Naugad Rural Municipality
16 Vyans Rural Municipality
16 Lekam Rural Municipality
16 Malikarjun Rural Municipality
16 Duhun Rural Municipality
16 Marma Rural Municipality
13 Navadurga Rural Municipality
13 Ganyapdhura Rural Municipality
13 Parashuram Municipality
13 Aalital Rural Municipality
13 Ajayameru Rural Municipality
13 Bhageshwor 

21 Tripurasundari Municipality
21 Mudkechula Rural Municipality
44 Khatyad Rural Municipality
44 Chhayanath Rara Municipality
44 Mugum Karmarong Rural Municipality
44 Soru Rural Municipality
61 Sharada Municipality
61 Chhatreshwori Rural Municipality
61 Bangad Kupinde Municipality
61 Triveni Rural Municipality
61 Kalimati Rural Municipality
61 Bagchaur Municipality
61 Kapurkot Rural Municipality
61 Darma Rural Municipality
61 Kumakh Rural Municipality
61 Siddhakumakh Rural Municipality
77 Chaurjahari Municipality
77 Aathbiskot Municipality
77 Sanibheri Rural Municipality
77 Triveni Rural Municipality
77 Banphikot Rural Municipality
77 Musikot Municipality
27 Kushe Rural Municipality
27 Junichande Rural Municipality
27 Bheri Municipality
27 Barekot Rural Municipality
27 Shivalaya Rural Municipality
27 Nalgad Municipality
27 Chhedagad Municipality
31 Khandachakra Municipality
31 Shuvakalika Rural Municipality
31 Palata Rural Municipality
31 Naraharinath Rural Municipality
31 Tilagupha Mu

23 Shahid Lakhan Rural Municipality
03 Nisikhola Rural Municipality
03 Kathekhola Rural Municipality
03 Galkot Municipality
03 Badigad Rural Municipality
03 Baglung Municipality
03 Bareng Rural Municipality
03 Tamankhola Rural Municipality
03 Dhorpatan Municipality
03 Tarakhola Rural Municipality
03 Jaimini Municipality
71 Kaligandaki Rural Municipality
71 Chapakot Municipality
71 Phedikhola Rural Municipality
71 Aandhikhola Rural Municipality
71 Biruwa Rural Municipality
71 Bhirkot Municipality
71 Waling Municipality
71 Galyang Municipality
71 Arjunchaupari Rural Municipality
71 Putalibazar Municipality
71 Harinas Rural Municipality
47 Devchuli Municipality
47 Bulingtar Rural Municipality
47 Kawasoti Municipality
47 Madhyabindu Municipality
47 Binayi Triveni Rural Municipality
47 Gaindakot Municipality
47 Hupsekot Rural Municipality
47 Baudikali Rural Municipality
46 Dhawalagiri Rural Municipality
46 Beni Municipality
46 Malika Rural Municipality
46 Annapurna Rural Municipality
46 Man

05 Durgathali Rural Municipality
05 Jayaprithvi Municipality
05 Masta Rural Municipality
05 Chhabis Pathibhera Rural Municipality
05 Saipal Rural Municipality
05 Khaptadchhanna Rural Municipality
32 Mahakali Municipality
32 Krishnapur Municipality
32 Bedkot Municipality
32 Shuklaphanta Municipality
32 Belauri Municipality
32 Laljhadi Rural Municipality
32 Punarbas Municipality
32 Beldandi Rural Municipality
32 Bhimdatta Municipality
69 Barahakshetra Municipality
69 Gadhi Rural Municipality
69 Ramdhuni Municipality
69 Barju Rural Municipality
69 Dewangunj Rural Municipality
69 Harinagar Rural Municipality
69 Bhokraha Narsingh Rural Municipality
69 Koshi Rural Municipality
69 Inaruwa Municipality
69 Itahari Sub-Metropolitan City
69 Dharan Sub-Metropolitan City
69 Duhabi Municipality
28 Kachankawal Rural Municipality
28 Shivasatakshi Municipality
28 Haldibari Rural Municipality
28 Kankai Municipality
28 Gaurigunj Rural Municipality
28 Bhadrapur Municipality
28 Damak Municipality
28 Jhapa 